In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from google.cloud import bigquery
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.environ["DESTINATION__BIGQUERY__CREDENTIALS__FILEPATH"]

client = bigquery.Client(project="de-zoomcamp-485314")

dataset_id = f"{client.project}.dlt_yellow_taxi_dataset"

dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"

dataset = client.create_dataset(dataset, exists_ok=True)
print("Dataset created:", dataset.dataset_id)

Dataset created: dlt_yellow_taxi_dataset


In [3]:
import requests
import pandas as pd
from io import BytesIO
import dlt

In [4]:
@dlt.source(name="rides")
def download_parquet():
    prefix = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata"

    for month in range(1, 7):
        file_name = f"yellow_tripdata_2024-0{month}"
        url = f"{prefix}_2024-0{month}.parquet"

        response = requests.get(url)
        response.raise_for_status()

        df = pd.read_parquet(BytesIO(response.content))
        yield dlt.resource(df, name=file_name)

In [ ]:
pipeline = dlt.pipeline(
    pipeline_name="rides_pipeline",
    destination="bigquery",
    dataset_name="dlt_yellow_taxi_dataset",
)

In [6]:
load_info = pipeline.run(
    download_parquet(),
    loader_file_format="parquet"
)

load_info

C:\Users\kimma\Documents\Projects\DE Zoomcamp\Mod 3\.venv\Lib\site-packages\google\cloud\bigquery\client.py:613: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(
C:\Users\kimma\Documents\Projects\DE Zoomcamp\Mod 3\.venv\Lib\site-packages\google\cloud\bigquery\client.py:613: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


LoadInfo(pipeline=<dlt.pipeline(pipeline_name='rides_pipeline', destination='bigquery', dataset_name='dlt_yellow_taxi_dataset', default_schema_name='rides', schema_names=['rides'], first_run=False, dev_mode=False, is_active=True, pipelines_dir='.dlt\\pipelines', working_dir='.dlt\\pipelines\\rides_pipeline')>, metrics={'1770644136.5038326': [{'started_at': DateTime(2026, 2, 9, 13, 35, 48, 230600, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2026, 2, 9, 13, 37, 24, 332961, tzinfo=Timezone('UTC')), 'job_metrics': {'yellow_tripdata_2024_01.6f32480cf4.parquet': LoadJobMetrics(job_id='yellow_tripdata_2024_01.6f32480cf4.parquet', file_path='C:\\Users\\kimma\\Documents\\Projects\\DE Zoomcamp\\Mod 3\\.dlt\\pipelines\\rides_pipeline\\load\\normalized\\1770644136.5038326\\started_jobs\\yellow_tripdata_2024_01.6f32480cf4.0.parquet', table_name='yellow_tripdata_2024_01', started_at=DateTime(2026, 2, 9, 13, 35, 59, 950981, tzinfo=Timezone('UTC')), finished_at=DateTime(2026, 2, 9, 13, 37, 11, 12